In [1]:
!pip install --quiet --progress-bar off pymatgen
!pip install --quiet --progress-bar off periodictable

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
cupy-cuda11x 11.0.0 requires numpy<1.26,>=1.20, but you have numpy 1.26.2 which is incompatible.


In [2]:
!git clone -q --single-branch --branch main https://github.com/Fung-Lab/ActiveStructOpt.git
%cd -q ActiveStructOpt
!pip install --quiet --progress-bar off -r requirements.txt
!pip install --quiet --progress-bar off -e .

  Preparing metadata (setup.py) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
notebook 6.5.5 requires pyzmq<25,>=17, but you have pyzmq 25.1.1 which is incompatible.
tensorflow 2.14.0 requires numpy>=1.23.5, but you have numpy 1.23.0 which is incompatible.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.8.0 which is incompatible.
  Preparing metadata (setup.py) ... done


In [3]:
import requests
from pymatgen.core.structure import Structure
from pymatgen.core.composition import Composition
import numpy as np
import matplotlib.pyplot as plt
import periodictable
from activestructopt.optimization.shared.constraints import lj_reject

MP_BASE_URL = "https://api.materialsproject.org/"

In [4]:
headers = {'accept': 'application/json', 'X-API-KEY': 'YOUR_API_KEY_HERE'}
structures = []
done = False
i = 0
while not done:
  query = {
  'theoretical': 'false',
  '_skip': i * 1000,
  '_limit': 1000,
  'nsites_max': 20,
  '_fields': 'structure,elements'
  }
  response = requests.get(MP_BASE_URL + "materials/summary",
                      params = query, headers = headers)
  done = len(response.json()['data']) < 1000
  i = i + 1
  structures.extend([Structure.from_dict(
      d['structure']) for d in response.json()['data']])

In [5]:
len(structures)

23270

In [6]:
rand_indices = np.random.choice(len(structures), 100, replace=False)

In [7]:
pristine_stuctures = [structures[i] for i in rand_indices]

In [8]:
perturbed_structures = [pristine_stuctures[i].copy() for i in range(len(pristine_stuctures))]
for i in range(len(pristine_stuctures)):
  rejected = True
  while rejected:
    new_structure = pristine_stuctures[i].copy()
    new_structure.perturb(0.5)
    rejected = lj_reject(new_structure)
  perturbed_structures[i] = new_structure.copy()

In [9]:
%cd -q /content
!mkdir pristine
!mkdir perturbed

In [10]:
for i in range(len(pristine_stuctures)):
  pristine_stuctures[i].to(filename=f"/content/pristine/{i}.cif")
for i in range(len(perturbed_structures)):
  perturbed_structures[i].to(filename=f"/content/perturbed/{i}.cif")

In [11]:
!zip -r pristine.zip pristine
!zip -r perturbed.zip perturbed

  adding: pristine/ (stored 0%)
  adding: pristine/40.cif (deflated 56%)
  adding: pristine/61.cif (deflated 63%)
  adding: pristine/36.cif (deflated 54%)
  adding: pristine/69.cif (deflated 67%)
  adding: pristine/55.cif (deflated 68%)
  adding: pristine/26.cif (deflated 67%)
  adding: pristine/17.cif (deflated 55%)
  adding: pristine/27.cif (deflated 60%)
  adding: pristine/85.cif (deflated 62%)
  adding: pristine/43.cif (deflated 60%)
  adding: pristine/94.cif (deflated 57%)
  adding: pristine/76.cif (deflated 56%)
  adding: pristine/80.cif (deflated 61%)
  adding: pristine/99.cif (deflated 51%)
  adding: pristine/21.cif (deflated 51%)
  adding: pristine/30.cif (deflated 57%)
  adding: pristine/32.cif (deflated 67%)
  adding: pristine/88.cif (deflated 62%)
  adding: pristine/38.cif (deflated 63%)
  adding: pristine/25.cif (deflated 59%)
  adding: pristine/96.cif (deflated 55%)
  adding: pristine/89.cif (deflated 60%)
  adding: pristine/53.cif (deflated 54%)
  adding: pristine/87.cif